In [14]:
# 1. Setup and initialization
import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.query import MetadataQuery

from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

# Initialize clients
wcd_url = os.environ["WCD_URL"]
wcd_api_key = os.environ["WCD_API_KEY"]

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,                                    
    auth_credentials=Auth.api_key(wcd_api_key),
)
openai_client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [15]:
# 2. Define the query
query = "H"

# 3. Get embedding for the query
response = openai_client.embeddings.create(
    model="text-embedding-3-large",
    input=query
)
query_embedding = response.data[0].embedding

In [16]:
# 4. Search for similar texts in Weaviate
collection_name = "Ebuka"

collection = client.collections.get(collection_name)
similar_texts = collection.query.near_vector(
    near_vector=query_embedding,
    limit=3,
    return_properties=["text"],
    return_metadata=MetadataQuery(distance=True)
)

# print("\nRetrieved contexts:")
# for i, doc in enumerate(similar_texts.objects, 1):
#     print(f"\n{i}. Distance: {doc.metadata.distance}")
#     print(f"Text: {doc.properties['text']}")
#     print("\n\n")

In [17]:
# 5. Prepare context for GPT
context_str = "\n\n---\n\n".join([doc.properties["text"] for doc in similar_texts.objects])

prompt = f"""Answer the question using ONLY the information provided in the context below. 
Do not add any general knowledge or information not contained in the context."

Context:
{context_str}

Question: {query}

Answer:"""

# 6. Generate answer using GPT-4
response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context."},
        {"role": "user", "content": prompt}
    ],
    temperature=0
)

answer = response.choices[0].message.content

# 7. Print results
# print(f"Prompt: {prompt} \n\n----------------------------------\n\n")
# print("Ebuka-------\n")
print(f"Question: {query}\n\nAnswer: {answer}")

Prompt: Answer the question using ONLY the information provided in the context below. 
Do not add any general knowledge or information not contained in the context."

Context:
Interior Power Outlets

The Cybertruck center console is equipped with two NEMA 5-20 120V power outlets capable of providing up to 20A combined. To enable power to the outlets, touch Controls > Outlets & Mods. Touch Enable Cabin Outlets on Entry to enable them by default when you enter Cybertruck.

If an outlet is not providing power as expected, see Troubleshooting AC Outlets on page 23.

NOTE: The 120V power outlets provide 20A maximum draw between both outlets. If multiple devices are plugged in and drawing power at the same time, power to each device may be reduced or interrupted.

NOTE: Power to AC outlets may be disabled in very low ambient temperatures.

NOTE: Power to AC outlets is disabled when an AC charging cable is connected to Cybertruck even if the vehicle is not actively charging.

Power to AC outl